In [17]:
'''
Authors: Yarala Hruthik Reddy, Kuruba Kiran Kumar, Surya Keesara
Title: GGH Cryptosystem Implementation on Image Files
'''

from PIL import Image
import random
import numpy as np
import base64
import binascii
import os
import time


# This function generates a private and public key

def keyGen(dimension):
    privateKey = []
    print("Searching for a private key")
    ratio = 1
    '''
    while True:
        privateKey = np.random.randint(-10, 10, size=(dimension,dimension))
        ratio = hadamardRatio(privateKey, dimension)
        if(.9 <= ratio <= 1):
            print(privateKey)
            break
    '''
    privateKey = np.identity(dimension)
    print(privateKey)

    print("Searching for a public key")
    while True:
        uniMod = randUniMod(dimension)
        temp = np.matmul(uniMod, privateKey)
        ratio = hadamardRatio(temp, dimension)
        if ratio <= .1:
            publicKey = temp
            break
    print(publicKey)

    return privateKey, publicKey, uniMod


# This function returns the Hadamard Ratio of a matrix.Time Complexity o(n)

def hadamardRatio(matrix, dimension): 
    detOfLattice = np.linalg.det(matrix)
    detOfLattice = detOfLattice if detOfLattice > 0 else -detOfLattice
    mult = 1
    for v in matrix:
        mult = mult * np.linalg.norm(v)
    hadRatio = (detOfLattice / mult) ** (1.0/dimension)
    return hadRatio


# This function returns a Random Unimodular matrix

def randUniMod(dimension):
    random_matrix = [[np.random.randint(-10, 10,)
                      for _ in range(dimension)] for _ in range(dimension)]
    upperTri = np.triu(random_matrix, 0)
    lowerTri = [[np.random.randint(-10, 10) if x <
                 y else 0 for x in range(dimension)] for y in range(dimension)]

    #Creating an upper trianglular and lower triangular matrices with diagonals as +1 or -1
    for r in range(len(upperTri)):
    	for c in range(len(upperTri)):
    		if(r == c):
    			if bool(random.getrandbits(1)):
    				upperTri[r][c] = 1
    				lowerTri[r][c] = 1
    			else:
    				upperTri[r][c] = -1
    				lowerTri[r][c] = -1
    uniModular = np.matmul(upperTri, lowerTri)
    return uniModular


# Converts images to black and white

def black_and_white(input_image_path, output_image_path):
   color_image = Image.open(input_image_path)
   bw = color_image.convert('L')
   bw.save(output_image_path)


# Loads images and copies the encoded image in a string

def loadImage(file_path):
    with open(file_path, "rb") as img_file:
        my_string = base64.b64encode(img_file.read())
    print("Image Loaded")
    return my_string


# Writes the encoded string to a file

def writeImageToString(filePath, string):
    newString = string.decode("utf-8")
    file = open(filePath, "w")
    file.write(str(newString))
    
    file.close()


# String to Image Converter

def writeStringToImage(filePath, imagePath):
    with open(filePath, "r") as f:
        string = f.read().replace("\n", "")
    imageData = base64.b64decode(string)
    with open(imagePath, "wb") as f:
        f.write(imageData)


# Writes text files

def writeTextBlocks(filePath, string):
    file = open(filePath, "w")
    file.write(str(string))

    file.close()


# Encoded text to Encrypted Ints 

def encodedToBinaryToEncrypted(seekVal):
    encoded = []
    with open("/content/drive/My Drive/CryptoProject/ImageString/imageString.txt", 'r') as f:
        encText = f.seek(seekVal)
        encText = f.read(10)
        for c in encText: 
            encoded.append(base64.b64encode(bytes(c, "utf-8")))

    binaryMessage = []
    for i in range(len(encoded)):
        binaryMessage.append(binascii.a2b_base64(encoded[i]))
    
    encryptedInts = []
    for i in range(len(encoded)):
        encryptedInts.append(int.from_bytes(binaryMessage[i], byteorder='little'))
    
    return encryptedInts


# Encryption Function

def encrypt(encryptedInts, publicKey):
    cypherText = []
    cypherText = np.matmul(encryptedInts, publicKey)

    #print("\n---------------------Cypher Array---------------------\n", cypherText)
    return cypherText


# Decryption Function

def decrypt(cypherText, privateKey, uniModular):
    A = privateKey
    x = cypherText
    BPRIME = np.linalg.inv(A)
    BB = np.matmul(BPRIME, x)
    uniModularInv = np.linalg.inv(uniModular)
    m = np.round(np.matmul(BB, uniModularInv)).astype(int)

    #print("\n---------------------Message Array---------------------\n", m)

    return m


# Misc Methods

# Writes decrypted message to a file

def writeDecryptedMessage(filePath, message):
    file = open(filePath, "a+")
    for i in range(len(message)):
        letter = chr(abs(message[i]))
        file.write(letter)

    file.close()

"""
# Writes encrypted message to a file
def showEncryptedMessage(filePath, message):
    file = open(filePath, "a+")
    for i in range(len(message)):
        letter = chr(abs(int(message[i])))
        file.write(letter)
    file.close()"""



# Writes Public key to a file

def writePublicKey(publicKey):
    file = open('/content/drive/My Drive/CryptoProject/PublicKey/ggh_block.txt', 'w')
    file.write("------BEGIN GGH PUBLIC KEY BLOCK -----\n")
    """
    print("length")
    print(len(publicKey))"""
    for row in range(len(publicKey)):
            for col in range(len(publicKey)):
                encoded = base64.b64encode(publicKey[row][col])
                file.write(str(encoded)[8:13])
            file.write("\n")

    file.write("\n--------END GGH PUBLIC KEY BLOCK -------")
    file.close()


# Writes the residue to the final message

def residueAdder(filePath,residueString):
    file = open(filePath, "a")
    for i in range(len(residueString)):
        letter = residueString[i]
        file.write(letter)

    file.close()


# Main Function

def main():
    """
    dirs = ['BWImages', 'Decrypted', 'Encrypted', 'ImageString', 'PublicKey', 'Residue']
    for i in dirs:
        if not os.path.exists(i):
            os.mkdir(i)
            print("Directory ", i,  " Created ")
        else:
            print("Directory ", i,  " already exists")"""
    start_time = time.time()
    alice = keyGen(10)#Time Complexity:-O(n2)
    writePublicKey(alice[1])#Time Complexity:-O(n2)

    cypherTextFile = []

    inputFileName =  "/content/drive/My Drive/CryptoProject/InputImages/8.jpg"
    
    OutputMessageFile =  "/content/drive/My Drive/CryptoProject/Decrypted/decryptedTextFile.txt"
    
    black_and_white(inputFileName, '/content/drive/My Drive/CryptoProject/BWImages/bwImage.jpg')
    
    stringEncoded = loadImage("/content/drive/My Drive/CryptoProject/BWImages/bwImage.jpg")
   
    writeImageToString("/content/drive/My Drive/CryptoProject/ImageString/imageString.txt", stringEncoded)
    print(len(stringEncoded))
    
    print("\n------------------------------Encrypting------------------------------\n")
    seekVal = 0
    while True:
        encryptedInts = encodedToBinaryToEncrypted(seekVal)
        if (seekVal > len(stringEncoded) - 10):
            residueString = stringEncoded[-(len(stringEncoded) - seekVal):]
            break
        else:
            seekVal = seekVal + 10
        bob = encrypt(encryptedInts, alice[1])
        cypherTextFile.append(bob)
    
    residueString = residueString.decode("utf-8")
    
    writeTextBlocks("/content/drive/My Drive/CryptoProject/Encrypted/cypherTextFile.txt", cypherTextFile)
    writeTextBlocks("/content/drive/My Drive/CryptoProject/Residue/residueText.txt", residueString)
    v=time.time() - start_time
    print("--- %s total encryption time  ---" % v)

    """
    print("\n------------------------------Encrypted Message Print------------------------------\n")
    seekVal = 0
    while True:
        if (seekVal >= len(cypherTextFile)):
            break
        else:
            showEncryptedMessage("Encrypted\\encryptedMessage.txt", cypherTextFile[seekVal])
       
    residueAdder("Encrypted\\encryptedMessage.txt", residueString)
    writeStringToImage("Encrypted\\encryptedMessage.txt", "Encrypted\\encryptedImage.jpg")"""
    
    
    
    print("\n------------------------------Decrypting------------------------------\n")
    start_time1 = time.time()#start time of decryption
    seekVal = 0
    decryptedTextFile = []
    while True:
        if (seekVal >= len(cypherTextFile)):
            break
        else:
            aliceReceives = decrypt(cypherTextFile[seekVal], alice[0], alice[2])
            decryptedTextFile.append(aliceReceives)
        seekVal = seekVal + 1
    
    writeTextBlocks("/content/drive/My Drive/CryptoProject/Decrypted/decryptedTextFile.txt", decryptedTextFile)
    
    print("\n------------------------------Decrypted Message Printing------------------------------\n")
    
    seekVal = 0
    while True:
        if (seekVal >= len(decryptedTextFile)):
            break
        else:
            writeDecryptedMessage(OutputMessageFile, decryptedTextFile[seekVal])
        seekVal = seekVal + 1
    
    residueAdder(OutputMessageFile,residueString)
    
    writeStringToImage(OutputMessageFile,"/content/drive/My Drive/CryptoProject/Decrypted/decryptedImage.jpg")
    
    print("--- %s total RUNNING    time  ---" %(time.time()-start_time))
    print("--- %s total DECRYPTING time  ---" %(time.time()-start_time1))
if __name__ == "__main__":
    main()

Searching for a private key
[[1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]]
Searching for a public key
[[ -65. -115.   69. -105.  -86.  -17. -131.  -39.   64.    8.]
 [  56.    1. -139.  -41. -112.  -61.   60.  -22.  -17.   -3.]
 [-116.  -70.   42.   44.  -19.   14.  -58.  -15.    1.    0.]
 [-100.  -47. -108.   26.  -70. -113.  103.  -53.  -56.   -9.]
 [-112.  -64.   27.  -63.  -84.  -42.  -65.  -46.   14.    1.]
 [ -73.  -49. -104.  -29.  -89. -109.   74.  -53.  -34.   -6.]
 [  70.  -14.  -48.   75.  112.   53.   63.   51.    0.    1.]
 [ -45.    2.  -36.  -11.  -60.  -54.   27.  -31.  -24.   -4.]
 [  13.  -38.   28.  -38.   14.    7.  -35.    2.   29.    4.]
 [   5.   -8.    7.   -7.    6.    4.   -8.    

In [0]:
!df -h

Filesystem      Size  Used Avail Use% Mounted on
overlay          69G   32G   34G  49% /
tmpfs            64M     0   64M   0% /dev
tmpfs           6.4G     0  6.4G   0% /sys/fs/cgroup
shm             5.8G     0  5.8G   0% /dev/shm
tmpfs           6.4G   16K  6.4G   1% /var/colab
/dev/sda1        75G   33G   43G  44% /opt/bin
tmpfs           6.4G     0  6.4G   0% /proc/acpi
tmpfs           6.4G     0  6.4G   0% /proc/scsi
tmpfs           6.4G     0  6.4G   0% /sys/firmware
drive           1.0E  6.9G  1.0E   1% /content/drive


In [0]:
!cat /proc/cpuinfo

processor	: 0
vendor_id	: GenuineIntel
cpu family	: 6
model		: 63
model name	: Intel(R) Xeon(R) CPU @ 2.30GHz
stepping	: 0
microcode	: 0x1
cpu MHz		: 2299.998
cache size	: 46080 KB
physical id	: 0
siblings	: 2
core id		: 0
cpu cores	: 1
apicid		: 0
initial apicid	: 0
fpu		: yes
fpu_exception	: yes
cpuid level	: 13
wp		: yes
flags		: fpu vme de pse tsc msr pae mce cx8 apic sep mtrr pge mca cmov pat pse36 clflush mmx fxsr sse sse2 ss ht syscall nx pdpe1gb rdtscp lm constant_tsc rep_good nopl xtopology nonstop_tsc cpuid tsc_known_freq pni pclmulqdq ssse3 fma cx16 pcid sse4_1 sse4_2 x2apic movbe popcnt aes xsave avx f16c rdrand hypervisor lahf_lm abm invpcid_single ssbd ibrs ibpb stibp fsgsbase tsc_adjust bmi1 avx2 smep bmi2 erms invpcid xsaveopt arat md_clear arch_capabilities
bugs		: cpu_meltdown spectre_v1 spectre_v2 spec_store_bypass l1tf mds swapgs itlb_multihit
bogomips	: 4599.99
clflush size	: 64
cache_alignment	: 64
address sizes	: 46 bits physical, 48 bits virtual
power management

In [0]:
!cat /proc/meminfo

MemTotal:       13333556 kB
MemFree:        10445968 kB
MemAvailable:   12436780 kB
Buffers:           75884 kB
Cached:          2072116 kB
SwapCached:            0 kB
Active:           750524 kB
Inactive:        1847292 kB
Active(anon):     429392 kB
Inactive(anon):      336 kB
Active(file):     321132 kB
Inactive(file):  1846956 kB
Unevictable:           0 kB
Mlocked:               0 kB
SwapTotal:             0 kB
SwapFree:              0 kB
Dirty:               316 kB
Writeback:             0 kB
AnonPages:        449664 kB
Mapped:           246928 kB
Shmem:               940 kB
Slab:             167068 kB
SReclaimable:     127232 kB
SUnreclaim:        39836 kB
KernelStack:        4352 kB
PageTables:         5936 kB
NFS_Unstable:          0 kB
Bounce:                0 kB
WritebackTmp:          0 kB
CommitLimit:     6666776 kB
Committed_AS:    2880576 kB
VmallocTotal:   34359738367 kB
VmallocUsed:           0 kB
VmallocChunk:          0 kB
Percpu:              920 kB
AnonHugePages:   